In [74]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import (plot_roc_curve, classification_report, roc_auc_score, average_precision_score,
confusion_matrix, plot_confusion_matrix, plot_precision_recall_curve)

In [59]:
data = pd.read_csv('InputedData.csv')
data

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,WindDir3pm_nan,RainToday_No,RainToday_Yes,RainToday_nan,RainTomorrow
0,2008/12/1,13.4,22.9,0.6,5.746016,7.198089,44.0,20.0,24.0,71.0,...,0,0,0,1,0,0,1,0,0,0
1,2008/12/2,7.4,25.1,0.0,5.862419,10.833726,44.0,4.0,22.0,44.0,...,0,0,0,0,1,0,1,0,0,0
2,2008/12/3,12.9,25.7,0.0,8.550697,11.735977,46.0,19.0,26.0,38.0,...,0,0,0,0,1,0,1,0,0,0
3,2008/12/4,9.2,28.0,0.0,6.849388,12.065107,24.0,11.0,9.0,45.0,...,0,0,0,0,0,0,1,0,0,0
4,2008/12/5,17.5,32.3,1.0,6.612760,5.543830,41.0,7.0,20.0,82.0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193287,2021/7/26,5.4,26.4,0.0,5.737812,10.888924,41.0,9.0,24.0,41.0,...,0,0,0,0,0,0,1,0,0,0
193288,2021/7/27,4.6,29.4,0.0,7.415299,12.543249,67.0,17.0,37.0,23.0,...,0,0,0,1,0,0,1,0,0,0
193289,2021/7/28,15.1,26.5,0.0,7.005784,5.392670,35.0,2.0,9.0,53.0,...,0,0,0,1,0,0,1,0,0,0
193290,2021/7/29,8.3,27.5,0.0,5.882616,9.225154,46.0,9.0,19.0,60.0,...,0,0,0,1,0,0,1,0,0,0


In [62]:
y = data['RainTomorrow']
date = data['Date']
data = data.drop(['Date', 'RainTomorrow'], axis = 1)

In [63]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size = 0.25, random_state = 1)

In [75]:
clf = DecisionTreeClassifier()
parameters = {'min_samples_leaf':range(100,5000,100),'max_depth':range(10,20,1),'criterion':['gini','entropy']}
clf_grid = RandomizedSearchCV(clf, parameters,cv=5,n_iter=300,scoring='recall',n_jobs=-1)
clf_grid_results = clf_grid.fit(X_train, y_train)
opt_para = clf_grid_results.best_params_
print("Following hyper-parameters give the best results: ", opt_para)
clf = clf_grid_results.best_estimator_
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred_prob = clf.predict_proba(X_test)[:,1]

Following hyper-parameters give the best results:  {'min_samples_leaf': 100, 'max_depth': 15, 'criterion': 'entropy'}


In [80]:
y_pred_train = clf.predict(X_train)
y_pred_prob_train = clf.predict_proba(X_train)[:,1]
print(roc_auc_score(y_train, y_pred_prob_train))

0.8904912878952096


In [78]:
print(classification_report(y_test, y_pred))
print(roc_auc_score(y_test, y_pred_prob))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91     37963
           1       0.70      0.51      0.59     10360

    accuracy                           0.85     48323
   macro avg       0.79      0.73      0.75     48323
weighted avg       0.84      0.85      0.84     48323

0.8627156883195659
